In [ ]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
def svd_reducedrank(A, k,  flag_bal):
    
    '''
    Decomposes matrix A by svd, performces reduced rank approximation of the rank k
    returs characteristic loads vector Lk and magnitudes Uk and reconstruction error
    
    Will performe load balancing if required
    
    inputs = [A, k,  flag_bal]
    outputs = [Uk, Lk, err_Ak]
    
    ------
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 03/03/2018  
    '''
    
    #Balacing   
    if flag_bal == 1:
#         S_bal = np.zeros([4,4])
        S_bal = np.zeros([A.shape[1],A.shape[1]])
        #compute normal of each coloumn and add it to S_bal diag
        S_bal = np.diag(1/np.linalg.norm(A,axis =0))
    else:
        S_bal = np.identity(A.shape[1])


    #Compute reduced rank internal characteristic loads
    U, s, VT= np.linalg.svd(np.dot(A,S_bal),full_matrices=1)

    if A.shape[0]>=A.shape[1]:
        S = np.zeros(A.shape)
    else:
        S = np.zeros([A.shape[0],A.shape[0]])

    S[:A.shape[1],:A.shape[1]] = np.diag(s)

    #Reduced rank
    Sk = S[:k,:k]
    Uk = U[:,:k]
    VTk = VT[:k,:]

    Lk_b = np.dot(Sk,VTk) #characteristic load
    Lk = np.dot(Lk_b,np.linalg.pinv(S_bal))
    
    #Error in reconstruction
    Ak = np.dot(Uk,Lk)
    err_Ak = (np.linalg.norm(A-Ak))*100/(np.linalg.norm(A,ord = 'fro'))
    
    return Uk, Lk, err_Ak

In [ ]:
def calculate_intersection_3D(loads,u_pe):
    
    '''
    Retuns intersection of the point with a plane formed by 3 points in a 3D space
    Notations used is from wiki https://www.wikiwand.com/en/Line%E2%80%93plane_intersection
    
    
    Inputs
    -------
    loads: characteristic load cartisian coordinates (array) 3D space
    u_pe: envelope points in 3D space
    
    Outputs
    -------
    I: array of intersection points 
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    
    '''
    
    hull = ConvexHull(u_pe)
    I = []
    for l_ind in np.arange(loads.shape[0]):
        
        for ind in hull.simplices:

            #Line segment 2 points
            la = np.array([[0],[0],[0]])
            lb = loads[[l_ind]].T


            #plane by 3 points
            p0 = u_pe[[ind[0]],:].T
            p1 = u_pe[[ind[1]],:].T
            p2 = u_pe[[ind[2]],:].T

            #assignments
            lab = lb - la
            p01 = p1 - p0
            p02 = p2 - p0

            temp_1 = np.hstack([-lab,p01,p02])
            temp_2 = la - p0

            para = np.dot(np.linalg.pinv(temp_1),temp_2) # parametric values [t,u,v]
            
            # if the determinant is zero then there is no unique solution i.e line is paralle or on the plane
            check_intersection_0 = np.linalg.det(temp_1)

            #If the value u+v<1, the point of intersection lies within the triangle formed by p0,p1,p2
            check_intersection_1 =  para[1]+para[2]
            
            #conditons
            # t >0: inersection is ahead of the ray 
            # u,v belongs to [0,1)
            # (u+v)<= 1 then the intersection lies within the triangle

            if (check_intersection_0 != 0) and (check_intersection_1 <= 1) and (para>=0).all():# and (para[1:]<=1).all():

                # intersection point
                intersection_p = la+lab*para[0]
                
        I.append(intersection_p.T)
    
    #Reshape the array
    I = np.asarray(I).reshape([loads.shape[0],3])
    return I

In [ ]:
def Fast_RF_estimation_3D(Uk,u_pe):
    
    '''
    Retuns critical RF for a set of loads and a performance envelope    
    
    Inputs
    -------
    Uk (array): characteristic loads in cartisian coordinates 3D space
    u_pe (array): envelope points in 3D space
    
    Outputs
    -------
    RF_pe: Critical RFs  
    
    Dependency
    ----------
    NA
    
    Author: Sriharsha Sheshanarayana sriharsha.sheshn@gmail.com
    Date: 07/03/2018
    '''
    I = calculate_intersection_3D(Uk,u_pe)

    r_allowable, temp_t,temp_p = cart2sph(I[:,0],I[:,1],I[:,2])
    r_actual, temp1_t,temp2_p = cart2sph(Uk[:,0],Uk[:,1],Uk[:,2])

    RF_pe = (r_allowable/r_actual).T

    return RF_pe,r_allowable